In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/n20-workshops/gti-fulbright-2025/data.csv")

In [ ]:
df

,id,created_at,text,location,city
0,T2013_00494,2013-02-14T19,Well put! @[REDACTED]: Many ways to knock #so...,"Berlin, Germany",Berlin
1,T2016_00056,2016-01-08T10,Sunrun on the run from Nevada in protest at ut...,"Berlin, Germany",Berlin
2,T2013_00929,2013-04-01T14,Interesting @[REDACTED] read on solar power in...,"Berlin, Germany",Berlin
3,T2018_00183,2018-01-15T10,#Solar News: Øresund Bridge connecting Sweden ...,"Berlin, Germany",Berlin
4,T2015_00330,2015-02-02T11,Jordan's solar tender bears fruit: Interest in...,"Berlin, Germany",Berlin
...,...,...,...,...,...
1068,T2021_00728,2021-04-29T09,Green electricity for green cars: RWE Supply a...,"Essen, Germany",Essen
1069,T2015_00890,2015-03-21T16,Rooftops on new commercial buildings in France...,"Essen, Germany",Essen
1070,T2019_00355,2019-02-12T09,We are excited to announce that we are partner...,"Essen, Germany",Essen
1071,T2021_01864,2021-10-18T10,Because the energy transition requires good co...,"Essen, Germany",Essen


In [ ]:
def add_sentiment(df, text_col="text", model_name="cardiffnlp/twitter-roberta-base-sentiment", batch_size=64):
    # Handle empty
    if df.empty:
        return df.copy()

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = 0 if torch.cuda.is_available() else -1

    pipe = TextClassificationPipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-classification",
        device=device,
        truncation=True,
        padding=True,
        max_length=256,
        return_all_scores=True
    )

    texts = df[text_col].fillna("").tolist()

    # Run in batches
    results = []
    for i in range(0, len(texts), batch_size):
        batch_out = pipe(texts[i:i+batch_size])
        results.extend(batch_out)

    # Map labels if using CardiffNLP
    label_map_cardiff = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}
    use_cardiff_map = "cardiffnlp/twitter-roberta-base-sentiment" in model_name.lower()

    # Determine final label names (for prob columns)
    raw_labels = [d["label"] for d in results[0]]
    final_labels = []
    for lab in raw_labels:
        if use_cardiff_map:
            final_labels.append(label_map_cardiff.get(lab, lab).lower())
        else:
            final_labels.append(lab.lower())

    # Collect outputs
    top_labels = []
    top_scores = {}
    for lab in final_labels:
        top_scores[lab] = []

    per_row_top_score = []
    per_row_top_label = []

    for r in results:
        # pick top
        best = max(r, key=lambda x: x["score"])
        best_label_raw = best["label"]
        if use_cardiff_map:
            best_label = label_map_cardiff.get(best_label_raw, best_label_raw).lower()
        else:
            best_label = best_label_raw.lower()
        per_row_top_label.append(best_label)
        per_row_top_score.append(best["score"])

        # probs per class
        row_probs = {}
        for item in r:
            raw = item["label"]
            if use_cardiff_map:
                norm = label_map_cardiff.get(raw, raw).lower()
            else:
                norm = raw.lower()
            row_probs[norm] = item["score"]

        for lab in final_labels:
            top_scores[lab].append(row_probs.get(lab, 0.0))

    out = df.copy()
    out["sentiment"] = per_row_top_label
    out["sent_confidence"] = per_row_top_score
    for lab in final_labels:
        out["prob_" + lab] = top_scores[lab]

    return out

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
# model_name = "oliverguhr/german-sentiment-bert"  # if your texts are German

df_with_sent_score = add_sentiment(df, text_col="text", model_name=model_name, batch_size=64)
df_sent = df_with_sent_score.copy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
df_sent

,id,created_at,text,location,city,sentiment,sent_confidence,prob_negative,prob_neutral,prob_positive
0,T2013_00494,2013-02-14T19,Well put! @[REDACTED]: Many ways to knock #so...,"Berlin, Germany",Berlin,neutral,0.531808,0.290170,0.531808,0.178022
1,T2016_00056,2016-01-08T10,Sunrun on the run from Nevada in protest at ut...,"Berlin, Germany",Berlin,neutral,0.718517,0.235295,0.718517,0.046188
2,T2013_00929,2013-04-01T14,Interesting @[REDACTED] read on solar power in...,"Berlin, Germany",Berlin,positive,0.880143,0.002265,0.117593,0.880143
3,T2018_00183,2018-01-15T10,#Solar News: Øresund Bridge connecting Sweden ...,"Berlin, Germany",Berlin,neutral,0.609252,0.003900,0.609252,0.386848
4,T2015_00330,2015-02-02T11,Jordan's solar tender bears fruit: Interest in...,"Berlin, Germany",Berlin,positive,0.663074,0.002475,0.334450,0.663074
...,...,...,...,...,...,...,...,...,...,...
1068,T2021_00728,2021-04-29T09,Green electricity for green cars: RWE Supply a...,"Essen, Germany",Essen,neutral,0.520357,0.003979,0.520357,0.475664
1069,T2015_00890,2015-03-21T16,Rooftops on new commercial buildings in France...,"Essen, Germany",Essen,neutral,0.914964,0.024130,0.914964,0.060906
1070,T2019_00355,2019-02-12T09,We are excited to announce that we are partner...,"Essen, Germany",Essen,positive,0.980092,0.000765,0.019143,0.980092
1071,T2021_01864,2021-10-18T10,Because the energy transition requires good co...,"Essen, Germany",Essen,positive,0.721204,0.006240,0.272557,0.721204


In [ ]:
df_sent.to_csv("", index = False)

# Mapping

In [ ]:
# ======================
# Part 1 — Blank GeoDataFrame
# ======================
import pandas as pd
import geopandas as gpd

city_coords = {
    "Berlin":      (52.5200, 13.4050),
    "Hamburg":     (53.5511,  9.9937),
    "Munich":      (48.1374, 11.5755),
    "Cologne":     (50.9375,  6.9603),
    "Frankfurt":   (50.1106,  8.6821),
    "Stuttgart":   (48.7784,  9.1806),
    "Dusseldorf":  (51.2277,  6.7735),
    "Leipzig":     (51.3397, 12.3731),
    "Dortmund":    (51.5136,  7.4653),
    "Essen":       (51.4556,  7.0116),
}

cities_df = pd.DataFrame(
    [{"city": c, "lat": lat, "lon": lon} for c, (lat, lon) in city_coords.items()]
)

gdf = gpd.GeoDataFrame(
    cities_df,
    geometry=gpd.points_from_xy(cities_df["lon"], cities_df["lat"]),
    crs="EPSG:4326"
)

In [ ]:
import folium

m = folium.Map(location=[51.1657, 10.4515], zoom_start=6, tiles="CartoDB positron")
m

In [ ]:
# ======================
# Part 2 — Compute tweet scores and merge with GeoDataFrame
# ======================
# Sentiment to numeric score mapping
sent_map = {"positive": 10, "neutral": 0, "negative": -10}

df_scored = df_sent.copy()
df_scored["sentiment"] = df_scored["sentiment"].str.lower()
df_scored["sent_score"] = df_scored["sentiment"].map(sent_map).fillna(0)

# Average score per city
city_avg = df_scored.groupby("city", as_index=False)["sent_score"].mean()

# Count number of tweets per sentiment type per city
city_counts = df_scored.groupby(["city", "sentiment"]).size().unstack(fill_value=0)

# Calculate proportions (%) for each sentiment
city_props = city_counts.div(city_counts.sum(axis=1), axis=0) * 100
city_props = city_props.reset_index()

# Merge average score and proportions
city_stats = pd.merge(city_avg, city_props, on="city", how="outer")

# Merge onto the GeoDataFrame (fill NaNs with 0 so all cities display)
gdf = gdf.merge(city_stats, on="city", how="left").fillna(0)

In [ ]:
# ======================
# Part 3 — Generate the final map
# ======================
import branca.colormap as cm

colormap = cm.LinearColormap(
    colors=["#d7191c", "#fdae61", "#ffffbf", "#a6d96a", "#1a9641"],
    vmin=-10, vmax=10
)
colormap.caption = "Average Sentiment (−10 negative … +10 positive)"
colormap.add_to(m)

for _, row in gdf.iterrows():
    score = float(row["sent_score"])
    neg_pct = row.get("negative", 0)
    neu_pct = row.get("neutral", 0)
    pos_pct = row.get("positive", 0)

    tooltip_text = (
        f"{row['city']} Sentiment Score: {score:.2f}<br>"
        f"Negative: {neg_pct:.1f}%<br>"
        f"Neutral: {neu_pct:.1f}%<br>"
        f"Positive: {pos_pct:.1f}%"
    )

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=9,
        color=colormap(score),
        fill=True,
        fill_opacity=0.85,
        weight=1,
        tooltip=tooltip_text
    ).add_to(m)

m